In [12]:
import pandas as pd
import re
import json
from datetime import datetime

print("Libraries imported successfully!")

Libraries imported successfully!


In [13]:
try:
    df = pd.read_excel('Pengadilan_Negeri_Surabaya.xlsx')
    print("File 'Pengadilan_Negeri_Surabaya.xlsx' berhasil dimuat.")
    print("\nInformasi Awal Dataset:")
    df.info()
except FileNotFoundError:
    print("Error: File 'Pengadilan_Negeri_Surabaya.xlsx' tidak ditemukan.")

# Tampilkan 5 baris pertama untuk melihat data mentah
print("\nData Mentah:")
display(df.head())

File 'Pengadilan_Negeri_Surabaya.xlsx' berhasil dimuat.

Informasi Awal Dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 25 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   Id                         55 non-null     object
 1   judul                      55 non-null     object
 2   nomor                      55 non-null     object
 3   irah irah                  55 non-null     object
 4   nama pengadilan negeri     55 non-null     object
 5   keterangan perkara         55 non-null     object
 6   identitas terdakwa         55 non-null     object
 7   penangkapan                55 non-null     object
 8   riwayat_perkara_penahanan  55 non-null     object
 9   tuntutan                   55 non-null     object
 10  dakwaan                    55 non-null     object
 11  bukti saksi                55 non-null     object
 12  bukti ahli                 55 non-null   

,Id,judul,nomor,irah irah,nama pengadilan negeri,keterangan perkara,identitas terdakwa,penangkapan,riwayat_perkara_penahanan,tuntutan,dakwaan,bukti saksi,bukti ahli,bukti terdakwa,bukti surat,petunjuk bb,fakta hukum,pertimbangan hukum,putusan,hari,tanggal,tahun,siapa yang memutus,panitera pengganti,tanda tangan majelis
0,1334_Pid.Sus_2025_PN_Sby.pdf,PUTUSAN,Nomor 1334/Pid.Sus/2025/PN Sby,DEMI KEADILAN BERDASARKAN KETUHANAN YANG MAHA ESA,Pengadilan Negeri Surabaya,Perkara Pidana dengan acara pemeriksaan biasa dalam tingkat pertama.,"[{""nama_lengkap"":""M. Noval Hadi Alias Sahrul Bin Abdul Rohman"",""tempat_lahir"":""Surabaya"",""umur_tanggal_lahir"":""21 tahun/18 Desember 2003"",""jenis_kelamin"":""Laki-laki"",""kebangsaan"":""Indonesia"",""tempat_tinggal"":""JI. Srengganan Dalam 1 No.2 Rt.002 Rw.007 Kel. Sidodadi Kec. Simokerto Surabaya"",""agama"":""Islam"",""pekerjaan"":""Belum/tidak bekerja""}]",Terdakwa ditangkap pada hari Senin tanggal 03 Maret 2025 sekira pukul 17.00 WIB di Jl. Srengganan Dalam 1 No. 7 Kel. Sidodadi Kec. Simokerto Surabaya.,Terdakwa M. Noval Hadi Alias Sahrul Bin Abdul Rohman ditahan dalam tahanan rutan oleh:\n1. Penyidik sejak tanggal 6 Maret 2025 sampai dengan tanggal 25 Maret 2025\n2. Penyidik Perpanjangan Oleh Penuntut Umum sejak tanggal 26 Maret 2025 sampai dengan tanggal 4 Mei 2025\n3. Penuntut Umum sejak tanggal 30 April 2025 sampai dengan tanggal 19 Mei 2025\n4. Penuntut Umum Perpanjangan Oleh Ketua Pengadilan Negeri (Pasal 25) sejak tanggal 20 Mei 2025 sampai dengan tanggal 18 Juni 2025\n5. Hakim Pengadilan Negeri sejak tanggal 16 Juni 2025 sampai dengan tanggal 15 Juli 2025\n6. Hakim Pengadilan Negeri Perpanjangan Oleh Ketua Pengadilan Negeri sejak tanggal 16 Juli 2025 sampai dengan tanggal 13 September 2025,"Penuntut Umum menuntut agar Majelis Hakim memutuskan:\n1. Menyatakan bahwa terdakwa, M. NOVAL HADI Alias SAHRUL Bin ABDUL ROHMAN, bersalah melakukan tindak pidana Narkotika sebagaimana diatur dan diancam pidana dalam pasal, 114 ayat (1) UU RI No. 35 tahun 2009 tentang Narkotika.\n2. Menjatuhkan pidana penjara terhadap terdakwa selama 6 (enam) tahun, dikurangi selama terdakwa berada dalam tahanan sementara, dengan perintah agar terdakwa tetap ditahan.\n3. Menjatuhkan Pidana denda sebesar Rp. 1.000.000.000,- (satu milyard rupiah) apabila tidak dibayar diganti dengan pidana selama 3 (tiga) bulan penjara.\n4. Menetapkan Barang bukti berupa: 1 (satu) bungkus plastik klip berisi kristal warna putih narkotika jenis sabu (sisa Labfor dikembalikan berat netto ± 0,076 gram) dan 1 (satu) dompet hijau dirampas untuk dimusnahkan. 1 (satu) Unit HP merk VIVO Y 16 warna gold dan Uang hasil penjualan sabu sebesar Rp. 450.000,00 dirampas untuk negara.\n5. Menetapkan supaya terdakwa dibebani membayar biaya perkara sebesar Rp. 2000,- (dua ribu rupiah).","PERTAMA\nBahwa terdakwa M. NOVAL HADI Alias SAHRUL Bin ABDUL ROHMAN pada hari Minggu tanggal 02 Maret 2025 atau pada waktu lain dalam Bulan Maret Tahun 2025 bertempat di Jl. Srengganan Dalam 1 No. 7 Kel. Sidodadi Kec. Simokerto Surabaya, tanpa hak atau melawan hukum, menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I. Perbuatan terdakwa sebagaimana diatur dan diancam pidana dalam pasal 114 ayat (1) UU RI No. 35 tahun 2009 tentang narkotika.\nATAU\nKEDUA\nBahwa terdakwa M. NOVAL HADI Alias SAHRUL Bin ABDUL ROHMAN pada hari Senin tanggal 03 Maret 2025 sekira pukul 17.00 WIB bertempat di Jl. Srengganan Dalam 1 No. 7 Kel. Sidodadi Kec. Simokerto Surabaya, tanpa hak atau melawan hukum, memiliki, menyimpan, menguasai atau menyediakan narkotika golongan I bukan tanaman. Perbuatan terdakwa sebagaimana diatur dan diancam pidana dalam pasal 112 ayat (1) UU RI No. 35 tahun 2009 tentang narkotika.","[""Saksi RIZA FAHLEFI: Menerangkan telah melakukan penangkapan terhadap terdakwa pada hari Senin, 03 Maret 2025, dan menemukan 1 poket sabu dengan berat netto 0,097 gram, dompet hijau, HP Viv

In [14]:
def parse_tanggal(date_str):
    """
    Mengubah berbagai format string tanggal menjadi objek datetime YYYY-MM-DD.
    Mampu menangani format seperti '2025-09-03 00:00:00' dan '10 Setember 2025'.
    """
    if pd.isna(date_str):
        return pd.NaT

    # Kamus untuk menerjemahkan bulan dalam Bahasa Indonesia
    month_map = {
        'januari': 'january', 'februari': 'february', 'maret': 'march', 'april': 'april',
        'mei': 'may', 'juni': 'june', 'juli': 'july', 'agustus': 'august',
        'september': 'september', 'oktober': 'october', 'november': 'november', 'desember': 'december',
        'setember': 'september' # Koreksi untuk typo umum
    }
    
    date_str = str(date_str).lower()
    for id_month, en_month in month_map.items():
        date_str = date_str.replace(id_month, en_month)
        
    try:
        # Coba parsing menggunakan pd.to_datetime yang sangat fleksibel
        return pd.to_datetime(date_str, errors='coerce').date()
    except (ValueError, TypeError):
        return pd.NaT

def parse_identitas_terdakwa(json_str):
    """
    Mengekstrak informasi dari string JSON di kolom 'identitas terdakwa'.
    Mengembalikan dictionary dengan informasi terdakwa pertama.
    """
    # Jika data kosong atau bukan string, kembalikan dictionary kosong
    if pd.isna(json_str) or not isinstance(json_str, str):
        return {}
        
    try:
        # Menghapus karakter di luar list JSON jika ada
        json_str = json_str.strip()
        if json_str.startswith('[') and json_str.endswith(']'):
            data = json.loads(json_str)
            if data:
                return data[0]  # Ambil data terdakwa pertama
    except json.JSONDecodeError:
        return {} # Kembalikan dict kosong jika JSON tidak valid
    return {}

def extract_numeric_from_string(text, keyword):
    """
    Mengekstrak angka pertama yang muncul setelah keyword tertentu.
    Contoh: mencari 'denda' dan mengambil '800000000'.
    """
    if not isinstance(text, str):
        return None
    
    # Cari keyword dan angka setelahnya
    match = re.search(f"{keyword}.*?rp[.,]?\\s*([\\d.,]+)", text.lower().replace('\n', ' '))
    if match:
        # Bersihkan angka dari titik atau koma
        num_str = match.group(1).replace('.', '').replace(',', '')
        return pd.to_numeric(num_str, errors='coerce')
    return None
    
print("Helper functions defined.")

Helper functions defined.


In [15]:
print("Starting data cleaning and standardization process...")

# 1. Standarisasi Kolom 'hari'
df['hari'] = df['hari'].str.lower().str.strip()

# 2. Parsing dan Standarisasi Kolom 'tanggal'
df['tanggal_putusan'] = df['tanggal'].apply(parse_tanggal)

# 3. Parsing Kolom 'identitas terdakwa'
# Terapkan fungsi dan ubah hasilnya menjadi DataFrame baru
identitas_df = df['identitas terdakwa'].apply(parse_identitas_terdakwa).apply(pd.Series)
# Ganti nama kolom agar lebih jelas
identitas_df = identitas_df.rename(columns=lambda col: f"terdakwa_{col}")
# Gabungkan kembali ke DataFrame utama
df = pd.concat([df, identitas_df], axis=1)

# 4. Ekstraksi Informasi dari Kolom 'Id'
id_parts = df['Id'].str.extract(r'(\d+)_([^_]+)_(\d+)_([^\.]+)\.pdf')
id_parts.columns = ['nomor_kasus', 'jenis_perkara', 'tahun_perkara', 'kode_pengadilan']
df = pd.concat([df, id_parts], axis=1)

# 5. Membersihkan Kolom Teks (contoh pada 'tuntutan' dan 'fakta hukum')
for col in ['tuntutan', 'fakta hukum', 'pertimbangan hukum', 'putusan', 'dakwaan']:
    if col in df.columns:
        df[col] = df[col].astype(str).apply(lambda x: re.sub(r'\s+', ' ', x).strip())

# 6. Ekstraksi Nilai Denda dari Kolom 'putusan' dan 'tuntutan'
df['denda_putusan_rp'] = df['putusan'].apply(lambda x: extract_numeric_from_string(x, 'denda'))
df['denda_tuntutan_rp'] = df['tuntutan'].apply(lambda x: extract_numeric_from_string(x, 'denda'))

print("Cleaning process finished.")

Starting data cleaning and standardization process...
Cleaning process finished.


In [16]:
print("Dataset setelah Cleaning dan Standarisasi:")
df.info()

# Pilih beberapa kolom penting untuk ditampilkan
kolom_tampil = [
    'Id', 'nomor_kasus', 'jenis_perkara', 'tahun_perkara',
    'hari', 'tanggal_putusan',
    'terdakwa_nama_lengkap', 'terdakwa_umur_tanggal_lahir', 'terdakwa_pekerjaan',
    'denda_putusan_rp', 'denda_tuntutan_rp'
]

# Menghapus kolom yang tidak ada jika terjadi error
kolom_tampil = [col for col in kolom_tampil if col in df.columns]

display(df[kolom_tampil].head())

Dataset setelah Cleaning dan Standarisasi:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 41 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Id                           55 non-null     object 
 1   judul                        55 non-null     object 
 2   nomor                        55 non-null     object 
 3   irah irah                    55 non-null     object 
 4   nama pengadilan negeri       55 non-null     object 
 5   keterangan perkara           55 non-null     object 
 6   identitas terdakwa           55 non-null     object 
 7   penangkapan                  55 non-null     object 
 8   riwayat_perkara_penahanan    55 non-null     object 
 9   tuntutan                     55 non-null     object 
 10  dakwaan                      55 non-null     object 
 11  bukti saksi                  55 non-null     object 
 12  bukti ahli                   55 non-n

,Id,nomor_kasus,jenis_perkara,tahun_perkara,hari,tanggal_putusan,terdakwa_nama_lengkap,terdakwa_umur_tanggal_lahir,terdakwa_pekerjaan,denda_putusan_rp,denda_tuntutan_rp
0,1334_Pid.Sus_2025_PN_Sby.pdf,1334,Pid.Sus,2025,rabu,2025-09-03,M. Noval Hadi Alias Sahrul Bin Abdul Rohman,21 tahun/18 Desember 2003,Belum/tidak bekerja,1.000000e+11,1000000000
1,1517_Pid.Sus_2025_PN_Sby.pdf,1517,Pid.Sus,2025,rabu,2025-09-10,Dewa Agira Panji Caroko Bin Dachlan Poernomo,24 tahun/17 Juni 2001,Karyawan swasta,8.000000e+08,800000000
2,1801_Pid.Sus_2025_PN_Sby.pdf,1801,Pid.Sus,2025,rabu,2025-10-08,ULFIANTO NOOR SATRIA TOHA ALIAS UPIK BIN ACHMAD TOHA RAHMAN (ALM),46 Tahun / 11 Desember 1978,Dagang,8.000000e+10,80000000000
3,1497_Pid.Sus_2025_PN_Sby.pdf,1497,Pid.Sus,2025,kamis,2025-09-11,Nur Rizal Ramadhani Bin Cornelis Jonathan,19/16 Oktober 2005,Karyawan Swasta,1.000000e+09,1000000000
4,1722_Pid.Sus_2025_PN_Sby.pdf,1722,Pid.Sus,2025,selasa,2025-09-23,Gilang Buana Bin Herry Soentoro,23 Th./14 Januari 2002,Swasta (ekspedisi),8.000000e+08,800000000


In [17]:
# Kolom yang akan dibuang (karena sudah diekstrak informasinya)
kolom_buang = ['identitas terdakwa', 'tanggal', 'tahun']

df_cleaned = df.drop(columns=kolom_buang, errors='ignore')

# Atur urutan kolom agar lebih logis
id_cols = ['Id', 'nomor_kasus', 'jenis_perkara', 'tahun_perkara', 'kode_pengadilan']
terdakwa_cols = sorted([col for col in df_cleaned.columns if col.startswith('terdakwa_')])
putusan_cols = ['hari', 'tanggal_putusan', 'denda_putusan_rp', 'denda_tuntutan_rp']
text_cols = [col for col in df_cleaned.columns if col not in id_cols + terdakwa_cols + putusan_cols]

new_order = id_cols + terdakwa_cols + putusan_cols + text_cols
df_cleaned = df_cleaned[new_order]


# Simpan ke file CSV
output_filename_csv = 'Pengadilan_Negeri_Surabaya_Cleaned.csv'
df_cleaned.to_csv(output_filename_csv, index=False)

print(f"Data yang sudah bersih telah disimpan ke '{output_filename_csv}'")
display(df_cleaned.head())

Data yang sudah bersih telah disimpan ke 'Pengadilan_Negeri_Surabaya_Cleaned.csv'


,Id,nomor_kasus,jenis_perkara,tahun_perkara,kode_pengadilan,terdakwa_agama,terdakwa_jenis_kelamin,terdakwa_kebangsaan,terdakwa_nama_lengkap,terdakwa_pekerjaan,terdakwa_pendidikan,terdakwa_tempat_lahir,terdakwa_tempat_tinggal,terdakwa_umur_tanggal_lahir,hari,tanggal_putusan,denda_putusan_rp,denda_tuntutan_rp,judul,nomor,irah irah,nama pengadilan negeri,keterangan perkara,penangkapan,riwayat_perkara_penahanan,tuntutan,dakwaan,bukti saksi,bukti ahli,bukti terdakwa,bukti surat,petunjuk bb,fakta hukum,pertimbangan hukum,putusan,siapa yang memutus,panitera pengganti,tanda tangan majelis
0,1334_Pid.Sus_2025_PN_Sby.pdf,1334,Pid.Sus,2025,PN_Sby,Islam,Laki-laki,Indonesia,M. Noval Hadi Alias Sahrul Bin Abdul Rohman,Belum/tidak bekerja,NaN,Surabaya,JI. Srengganan Dalam 1 No.2 Rt.002 Rw.007 Kel. Sidodadi Kec. Simokerto Surabaya,21 tahun/18 Desember 2003,rabu,2025-09-03,1.000000e+11,1000000000,PUTUSAN,Nomor 1334/Pid.Sus/2025/PN Sby,DEMI KEADILAN BERDASARKAN KETUHANAN YANG MAHA ESA,Pengadilan Negeri Surabaya,Perkara Pidana dengan acara pemeriksaan biasa dalam tingkat pertama.,Terdakwa ditangkap pada hari Senin tanggal 03 Maret 2025 sekira pukul 17.00 WIB di Jl. Srengganan Dalam 1 No. 7 Kel. Sidodadi Kec. Simokerto Surabaya.,Terdakwa M. Noval Hadi Alias Sahrul Bin Abdul Rohman ditahan dalam tahanan rutan oleh:\n1. Penyidik sejak tanggal 6 Maret 2025 sampai dengan tanggal 25 Maret 2025\n2. Penyidik Perpanjangan Oleh Penuntut Umum sejak tanggal 26 Maret 2025 sampai dengan tanggal 4 Mei 2025\n3. Penuntut Umum sejak tanggal 30 April 2025 sampai dengan tanggal 19 Mei 2025\n4. Penuntut Umum Perpanjangan Oleh Ketua Pengadilan Negeri (Pasal 25) sejak tanggal 20 Mei 2025 sampai dengan tanggal 18 Juni 2025\n5. Hakim Pengadilan Negeri sejak tanggal 16 Juni 2025 sampai dengan tanggal 15 Juli 2025\n6. Hakim Pengadilan Negeri Perpanjangan Oleh Ketua Pengadilan Negeri sejak tanggal 16 Juli 2025 sampai dengan tanggal 13 September 2025,"Penuntut Umum menuntut agar Majelis Hakim memutuskan: 1. Menyatakan bahwa terdakwa, M. NOVAL HADI Alias SAHRUL Bin ABDUL ROHMAN, bersalah melakukan tindak pidana Narkotika sebagaimana diatur dan diancam pidana dalam pasal, 114 ayat (1) UU RI No. 35 tahun 2009 tentang Narkotika. 2. Menjatuhkan pidana penjara terhadap terdakwa selama 6 (enam) tahun, dikurangi selama terdakwa berada dalam tahanan sementara, dengan perintah agar terdakwa tetap ditahan. 3. Menjatuhkan Pidana denda sebesar Rp. 1.000.000.000,- (satu milyard rupiah) apabila tidak dibayar diganti dengan pidana selama 3 (tiga) bulan penjara. 4. Menetapkan Barang bukti berupa: 1 (satu) bungkus plastik klip berisi kristal warna putih narkotika jenis sabu (sisa Labfor dikembalikan berat netto ± 0,076 gram) dan 1 (satu) dompet hijau dirampas untuk dimusnahkan. 1 (satu) Unit HP merk VIVO Y 16 warna gold dan Uang hasil penjualan sabu sebesar Rp. 450.000,00 dirampas untuk negara. 5. Menetapkan supaya terdakwa dibebani membayar biaya perkara sebesar Rp. 2000,- (dua ribu rupiah).","PERTAMA Bahwa terdakwa M. NOVAL HADI Alias SAHRUL Bin ABDUL ROHMAN pada hari Minggu tanggal 02 Maret 2025 atau pada waktu lain dalam Bulan Maret Tahun 2025 bertempat di Jl. Srengganan Dalam 1 No. 7 Kel. Sidodadi Kec. Simokerto Surabaya, tanpa hak atau melawan hukum, menawarkan untuk dijual, menjual, membeli, menerima, menjadi perantara dalam jual beli, menukar, atau menyerahkan Narkotika Golongan I. Perbuatan terdakwa sebagaimana diatur dan diancam pidana dalam pasal 114 ayat (1) UU RI No. 35 tahun 2009 tentang narkotika. ATAU KEDUA Bahwa terdakwa M. NOVAL HADI Alias SAHRUL Bin ABDUL ROHMAN pada hari Senin tanggal 03 Maret 2025 sekira pukul 17.00 WIB bertempat di Jl. Srengganan Dalam 1 No. 7 Kel. Sidodadi Kec. Simokerto Surabaya, tanpa hak atau melawan hukum, memiliki, menyimpan, menguasai atau menyediakan narkotika golongan I bukan tanaman. Perbuatan terdakwa sebagaimana diatur dan diancam pidana dalam pasal 112 ayat (1) UU RI No. 35 tahun 2009 tentang narkotika.","[""Saksi RIZA FAHLEFI: Menerangkan 